This notebook can be used for selecting the optimal image augmenations with the optuna package

## Install MMSegmentation and other packages, connect to gdrive
This step may take several minutes. 

We use PyTorch 1.10 and CUDA 11.1 for this tutorial. You may install other versions by change the version number in pip install command. 

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# Install PyTorch
# !pip install pytorch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 cudatoolkit==11.3 
!pip install pytorch
# Install mim
!pip install -U openmim
# Install mmengine
# !mim install mmengine - #this version 0.7.2 seems to cause an error, possibly already resolved
!mim install 'mmengine==0.7.3' #was 0.7.0 when successful
# !mim install mmengine
# Install MMCV
!mim install 'mmcv >= 2.0.0rc1'
!pip install "mmdet>=3.0.0rc4" #required for mask2former
!pip install "mmcls>=1.0.0rc0" #required for poolformer

#optuna
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

Mounted at /content/gdrive/


In [ ]:
#clone mmsegmentation repo
%cd /content
!rm -rf mmsegmentation
!git clone --branch dev-1.x --single-branch https://github.com/open-mmlab/mmsegmentation.git
# !git clone https://github.com/open-mmlab/mmsegmentation.git
# mmseg_folder = ~/content/gdrive/MyDrive/CaresAI/Code/segmentation
%cd mmsegmentation
!pip install -e .

/content
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 13307, done.
remote: Counting objects: 100% (505/505), done.
remote: Compressing objects: 100% (344/344), done.
remote: Total 13307 (delta 198), reused 329 (delta 151), pack-reused 12802
Receiving objects: 100% (13307/13307), 19.50 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (9717/9717), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation


In [ ]:
# #clone mmengine repo
# %cd /content
# !rm -rf mmengine
# # !git clone --branch 1.x --single-branch https://github.com/open-mmlab/mmsegmentation.git
# !git clone https://github.com/open-mmlab/mmengine.git
# # mmseg_folder = ~/content/gdrive/MyDrive/CaresAI/Code/segmentation
# %cd mmengine
# !pip install -e .
# # %cd /content/gdrive

We run an basic inference model to check the enviroment is configured corectly

Expected result:
After running demo cell, a segmentation prediction image is output to result.jpg

In [ ]:
#checking env is correct
# !mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest .

In [ ]:
# !python3 demo/image_demo.py demo/demo.png configs/pspnet/pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth --device cuda:0 --out-file result.jpg

Import needed packages:

In [ ]:
# Pytorch installation
import torch, torchvision
import torch.cuda.amp

#import mmseg
import mmseg

# # MMSegmentation installation
import mmcv
import mmengine
import mmdet

In [ ]:
#other
import optuna
import albumentations
import numpy as np
import os
import typing
from typing import Callable
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import seaborn as sns

In [ ]:
print(f'torch version: {torch.__version__}')
print(f'check if cuda is available: {torch.cuda.is_available()}')
print(f'mmcv_version: {mmcv.__version__}')
print(f'mmengine_version: {mmengine.__version__}')
print(f'mmsegmentation_version: {mmseg.__version__}')

torch version: 2.0.0+cu118
check if cuda is available: True
mmcv_version: 2.0.0
mmengine_version: 0.7.3
mmsegmentation_version: 1.0.0


## Creating train/val splits

https://bcsegmentation.grand-challenge.org/Baseline/

Slides from these institutes were not used in training the final segmentation model on the core set. They were used as an unseen testing set to report accuracy: OL, LL, E2, EW, GM, and S3.  <- Please use this testing set to reproduce the accuracy results from the table above.

In [ ]:
#set paths to images and masks
data_root = '/content/gdrive/MyDrive/BCSS_dataset'

use_patched = True
use_3d_masks = False
if (use_patched == True) and (use_3d_masks==False):
  img_dir_path = os.path.join(data_root,'patched_images_masks_2d_6_class','images')
  ann_dir_path = os.path.join(data_root,'patched_images_masks_2d_6_class','masks')
elif (use_patched == False) and (use_3d_masks==True):
  img_dir_path = img_dir_path = prep_dataset_obj.orig_images_folder_path
  ann_dir_path = os.path.join(data_root,'new_masks_3d_6_class')
elif (use_patched == False) and (use_3d_masks==False):
  img_dir_path = os.path.join(data_root,'images')
  ann_dir_path = os.path.join(data_root,'new_masks_2d_6_class')
print(img_dir_path)
print(ann_dir_path)

/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/images
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/masks


In [ ]:
split_dir = 'splits'
split_path = os.path.join(data_root, split_dir)
if not os.path.exists(split_path):
  os.makedirs(split_path)

#get list of filenames
filename_list = [os.path.splitext(filename)[0] for filename in os.listdir(ann_dir_path)]

train_files = []
val_files = []
test_files = []
val_labs = ['OL', 'LL', 'EW', 'GM','S3']
test_lab = 'E2' #randomly selected from validation/test sets
val_counter = dict.fromkeys(val_labs, 0) #start a counter to count files
test_counter = 0

#loop through filenames and assign into train/val/test set as needed
for filename in filename_list:
  lab_name = filename.split('-')[1]
  if lab_name in val_labs:
    val_counter[lab_name] = val_counter[lab_name] + 1
    val_files.append(filename)
  elif lab_name in test_lab:
    test_counter = test_counter+1
    test_files.append(filename)
  else:
    train_files.append(filename)

train_files = sorted(train_files)
val_files = sorted(val_files)

num_train_imgs = len(train_files)
num_val_imgs = len(val_files)
num_test_imgs = len(test_files)
total_imgs = num_train_imgs + num_val_imgs + num_test_imgs
print(f"num train files: {num_train_imgs} , % of total: {(num_train_imgs/total_imgs):.3f}")
print(f"num val files: {num_val_imgs} , % of total: {(num_val_imgs/total_imgs):.3f}")
print(f"num test files: {num_test_imgs} , % of total: {(num_test_imgs/total_imgs):.3f}")

print(val_counter)

with open(os.path.join(data_root, split_dir, 'train.txt'), 'w') as f:
  f.writelines(line + '\n' for line in train_files)

with open(os.path.join(data_root, split_dir, 'val.txt'), 'w') as f:
  f.writelines(line + '\n' for line in val_files) #choose half of val files to speed up validation step during optuna tuning

num train files: 3318 , % of total: 0.690
num val files: 974 , % of total: 0.202
num test files: 520 , % of total: 0.108
{'OL': 188, 'LL': 62, 'EW': 441, 'GM': 249, 'S3': 34}


In [ ]:
# #testing mask shape
# test_mask = ann_dir_path+'/' + train_files[0].split('/')[-1]+'.png'
# test_mask_img = Image.open(test_mask)
# test_array = np.asarray(test_mask_img)
# print(np.shape(test_array))

## Train a semantic segmentation model on a new dataset

To train on a customized dataset, the following steps are necessary. 
1. Add a new dataset class. 
2. Create a config file accordingly. 
3. Perform training and evaluation. 

### Add a new dataset

Datasets in MMSegmentation require image and semantic segmentation maps to be placed in folders with the same prefix. To support a new dataset, we may need to modify the original file structure. 

In this tutorial, we give an example of converting the dataset. You may refer to [docs](https://github.com/open-mmlab/mmsegmentation/blob/master/docs/en/tutorials/customize_datasets.md#customize-datasets-by-reorganizing-data) for details about dataset reorganization. 

We use [Stanford Background Dataset](http://dags.stanford.edu/projects/scenedataset.html) as an example. The dataset contains 715 images chosen from existing public datasets [LabelMe](http://labelme.csail.mit.edu), [MSRC](http://research.microsoft.com/en-us/projects/objectclassrecognition), [PASCAL VOC](http://pascallin.ecs.soton.ac.uk/challenges/VOC) and [Geometric Context](http://www.cs.illinois.edu/homes/dhoiem/). Images from these datasets are mainly outdoor scenes, each containing approximately 320-by-240 pixels. 
In this tutorial, we use the region annotations as labels. There are 8 classes in total, i.e. sky, tree, road, grass, water, building, mountain, and foreground object. 

In [ ]:
# download and unzip
# !wget http://dags.stanford.edu/data/iccv09Data.tar.gz -O stanford_background.tar.gz
# !tar xf stanford_background.tar.gz

## Creating dataset object

After downloading the data, we need to implement `load_annotations` function in the new dataset class `StanfordBackgroundDataset`.

In [ ]:
%%script false --no-raise-error
#example
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class StanfordBackgroundDataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

    

https://github.com/open-mmlab/mmsegmentation/blob/104429be008d84cd4ca7f162e2fd04f93c77e8ad/docs/en/tutorials/customize_datasets.md

https://github.com/open-mmlab/mmsegmentation/blob/5b438f45b6134368d2c1717446b954a1457743f0/mmseg/datasets/custom.py

In [ ]:
class_names_new = ['outside_roi','tumor','stroma','inflammatory','necrosis','other']
classes_new_int = [0,1,2,3,4,5]
# convert dataset annotation to semantic segmentation map
palette = sns.color_palette('tab20', len(classes_new_int))                                            
palette_array = np.asarray(palette)*255
# roll array so orange is tumour 
palette_array = np.roll(palette_array, -2,axis=0)
palette_array #shows palette

array([[255., 127.,  14.],
       [255., 187., 120.],
       [ 44., 160.,  44.],
       [152., 223., 138.],
       [ 31., 119., 180.],
       [174., 199., 232.]])

In [ ]:
#cancer dataset
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

@DATASETS.register_module()
class CancerDataset(BaseSegDataset):
  METAINFO = dict(classes = class_names_new, palette = palette_array)
  def __init__(self, **kwargs):
    super().__init__(img_suffix='.png', 
                     seg_map_suffix='.png', 
                     **kwargs)
  
  # def evaluate(self,
  #               results,
  #               metric='mDice',
  #               logger=None,
  #               gt_seg_maps=None,
  #               **kwargs):
    
  #   with torch.no_grad():
  #     metric='mDice'
  #     super().evaluate(results,
  #                     metric,
  #                     logger,
  #                     gt_seg_maps)

# cancerDataset_obj = CancerDataset()

### Create a config file

We now need to create a config file which defines how the data is loaded and the parameters for training and validating the model

In [ ]:
%cd /content

/content


In [ ]:
#download the checkpoints for the model of interest
current_model_path = os.path.join('mmsegmentation/checkpoints','current_model')
!rm $current_model_path -r

#segformer
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b0_512x512_160k_ade20k/segformer_mit-b0_512x512_160k_ade20k_20210726_101530-8ffa8fda.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b1_512x512_160k_ade20k/segformer_mit-b1_512x512_160k_ade20k_20210726_112106-d70e859d.pth -P $current_model_path
!wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b2_512x512_160k_ade20k/segformer_mit-b2_512x512_160k_ade20k_20210726_112103-cbd414ac.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b3_512x512_160k_ade20k/segformer_mit-b3_512x512_160k_ade20k_20210726_081410-962b98d2.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b4_512x512_160k_ade20k/segformer_mit-b4_512x512_160k_ade20k_20210728_183055-7f509d7d.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_512x512_160k_ade20k/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth -P $current_model_path

#mask2former
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_r50_8xb2-160k_ade20k-512x512/mask2former_r50_8xb2-160k_ade20k-512x512_20221204_000055-2d1f55f1.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_r101_8xb2-160k_ade20k-512x512/mask2former_r101_8xb2-160k_ade20k-512x512_20221203_233905-b7135890.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_swin-t_8xb2-160k_ade20k-512x512/mask2former_swin-t_8xb2-160k_ade20k-512x512_20221203_234230-7d64e5dd.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_swin-s_8xb2-160k_ade20k-512x512/mask2former_swin-s_8xb2-160k_ade20k-512x512_20221204_143905-e715144e.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_swin-b-in1k-384x384-pre_8xb2-160k_ade20k-640x640/mask2former_swin-b-in1k-384x384-pre_8xb2-160k_ade20k-640x640_20221129_125118-a4a086d2.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_swin-b-in22k-384x384-pre_8xb2-160k_ade20k-640x640/mask2former_swin-b-in22k-384x384-pre_8xb2-160k_ade20k-640x640_20221203_235230-7ec0f569.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/mask2former/mask2former_r50_8xb2-90k_cityscapes-512x1024/mask2former_r50_8xb2-90k_cityscapes-512x1024_20221202_140802-ffd9d750.pth -P $current_model_path

#poolformer
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/poolformer/fpn_poolformer_s12_8x4_512x512_40k_ade20k/fpn_poolformer_s12_8x4_512x512_40k_ade20k_20220501_115154-b5aa2f49.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/poolformer/fpn_poolformer_s24_8x4_512x512_40k_ade20k/fpn_poolformer_s24_8x4_512x512_40k_ade20k_20220503_222049-394a7cf7.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/poolformer/fpn_poolformer_s36_8x4_512x512_40k_ade20k/fpn_poolformer_s36_8x4_512x512_40k_ade20k_20220501_151122-b47e607d.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/poolformer/fpn_poolformer_m36_8x4_512x512_40k_ade20k/fpn_poolformer_m36_8x4_512x512_40k_ade20k_20220501_164230-3dc83921.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/poolformer/fpn_poolformer_m48_8x4_512x512_40k_ade20k/fpn_poolformer_m48_8x4_512x512_40k_ade20k_20220504_003923-64168d3b.pth -P $current_model_path

#convnext
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/convnext/upernet_convnext_tiny_fp16_512x512_160k_ade20k/upernet_convnext_tiny_fp16_512x512_160k_ade20k_20220227_124553-cad485de.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/convnext/upernet_convnext_small_fp16_512x512_160k_ade20k/upernet_convnext_small_fp16_512x512_160k_ade20k_20220227_131208-1b1e394f.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/convnext/upernet_convnext_base_fp16_512x512_160k_ade20k/upernet_convnext_base_fp16_512x512_160k_ade20k_20220227_181227-02a24fc6.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/fcn/fcn_r50-d8_512x512_80k_ade20k/fcn_r50-d8_512x512_80k_ade20k_20200614_144016-f8ac5082.pth -P $current_model_path
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/fcn/fcn_r101-d8_512x512_80k_ade20k/fcn_r101-d8_512x512_80k_ade20k_20200615_014143-bc1809f7.pth -P $current_model_path

checkpoint_path = os.path.join(current_model_path,os.listdir(current_model_path)[0])
print(f'checkpoint path: {checkpoint_path}')

rm: cannot remove 'mmsegmentation/checkpoints/current_model': No such file or directory
--2023-05-17 13:05:33--  https://download.openmmlab.com/mmsegmentation/v0.5/poolformer/fpn_poolformer_m48_8x4_512x512_40k_ade20k/fpn_poolformer_m48_8x4_512x512_40k_ade20k_20220504_003923-64168d3b.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 192.169.122.217, 192.169.122.210, 192.169.122.211, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|192.169.122.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308561423 (294M) [application/octet-stream]
Saving to: ‘mmsegmentation/checkpoints/current_model/fpn_poolformer_m48_8x4_512x512_40k_ade20k_20220504_003923-64168d3b.pth’

fpn_poolformer_m48_ 100%[===================>] 294.27M  92.3MB/s    in 3.4s    

2023-05-17 13:05:37 (86.5 MB/s) - ‘mmsegmentation/checkpoints/current_model/fpn_poolformer_m48_8x4_512x512_40k_ade20k_20220504_003923-64168d3b.pth’ saved [308561423/308561423]

checkpoint path:

In [ ]:
#import relevant config file for model of interest
from mmengine import Config
#segformer models
# segform_b0_ade = '/content/mmsegmentation/configs/segformer/segformer_mit-b0_8xb2-160k_ade20k-512x512.py'
# segform_b1_ade = '/content/mmsegmentation/configs/segformer/segformer_mit-b1_8xb2-160k_ade20k-512x512.py'
segform_b2_ade = '/content/mmsegmentation/configs/segformer/segformer_mit-b2_8xb2-160k_ade20k-512x512.py'
# segform_b3_ade = '/content/mmsegmentation/configs/segformer/segformer_mit-b3_8xb2-160k_ade20k-512x512.py'
# segform_b4_ade = '/content/mmsegmentation/configs/segformer/segformer_mit-b4_8xb2-160k_ade20k-512x512.py'
# segform_b5_ade = '/content/mmsegmentation/configs/segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py'
# segform_b4_city = '/content/mmsegmentation/configs/segformer/segformer_mit-b4_8xb1-160k_cityscapes-1024x1024.py'


#mask2former models
# mask2form_r50 = '/content/mmsegmentation/configs/mask2former/mask2former_r50_8xb2-160k_ade20k-512x512.py'
# mask2form_r101_ade = '/content/mmsegmentation/configs/mask2former/mask2former_r101_8xb2-160k_ade20k-512x512.py'
# mask2form_swin_t_ade = '/content/mmsegmentation/configs/mask2former/mask2former_swin-t_8xb2-160k_ade20k-512x512.py'
# mask2form_swin_s_ade = '/content/mmsegmentation/configs/mask2former/mask2former_swin-s_8xb2-160k_ade20k-512x512.py'
# mask2form_swin_b_ade = '/content/mmsegmentation/configs/mask2former/mask2former_swin-b-in1k-384x384-pre_8xb2-160k_ade20k-640x640.py'
# mask2form_swin_b2_ade = '/content/mmsegmentation/configs/mask2former/mask2former_swin-b-in22k-384x384-pre_8xb2-160k_ade20k-640x640.py'
# mask2form_r50_city = '/content/mmsegmentation/configs/mask2former/mask2former_r50_8xb2-90k_cityscapes-512x1024.py'

#poolformer
# poolform_s12_ade = '/content/mmsegmentation/configs/poolformer/fpn_poolformer_s12_8xb4-40k_ade20k-512x512.py'
# poolform_s24_ade = '/content/mmsegmentation/configs/poolformer/fpn_poolformer_s24_8xb4-40k_ade20k-512x512.py'
# poolform_s36_ade = '/content/mmsegmentation/configs/poolformer/fpn_poolformer_s36_8x4_512x512_40k_ade20k.py'
# poolform_m36_ade ='/content/mmsegmentation/configs/poolformer/fpn_poolformer_m36_8xb4-40k_ade20k-512x512.py'
poolform_m48_ade = '/content/mmsegmentation/configs/poolformer/fpn_poolformer_m48_8xb4-40k_ade20k-512x512.py'

#convnext
# convnext_t = '/content/mmsegmentation/configs/convnext/convnext-tiny_upernet_8xb2-amp-160k_ade20k-512x512.py'
# convnext_s = '/content/mmsegmentation/configs/convnext/convnext-small_upernet_8xb2-amp-160k_ade20k-512x512.py'
# convnext_b = '/content/mmsegmentation/configs/convnext/convnext-base_upernet_8xb2-amp-160k_ade20k-512x512.py'

#fcn
# fcn_r50 = '/content/mmsegmentation/configs/fcn/fcn_r50-d8_4xb4-80k_ade20k-512x512.py'
# fcn_r101 = '/content/mmsegmentation/configs/fcn/fcn_r101-d8_4xb4-80k_ade20k-512x512.py'

model_to_use = poolform_m48_ade
cfg_orig = Config.fromfile(model_to_use)
cfg = Config.fromfile(model_to_use)

Example config fle from tutorial:

https://github.com/open-mmlab/mmsegmentation/blob/104429be008d84cd4ca7f162e2fd04f93c77e8ad/docs/en/tutorials/data_pipeline.md

Config file for cancer dataset:

N.b if changing options in the config file, it seems necessary to reload the config file by rerunning the above command 
"cfg = Config.fromfile("

In [ ]:
print(f'Config:\n{cfg_orig.pretty_text}')
# cfg_orig.train_dataloadersegform_b5_ade

Config:
norm_cfg = dict(type='SyncBN', requires_grad=True)
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/poolformer/poolformer-m48_3rdparty_32xb128_in1k_20220414-9378f3eb.pth'
custom_imports = dict(imports=['mmcls.models'], allow_failed_imports=False)
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=(512, 512))
model = dict(
    type='EncoderDecoder',
    data_preprocessor=dict(
        type='SegDataPreProcessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_val=0,
        seg_pad_val=255,
        size=(512, 512)),
    backbone=dict(
        type='mmcls.PoolFormer',
        arch='m48',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/poolformer/poolformer-m48_3r

Create config file to control data loading and model training process:

In [ ]:
# DEBUG: Detailed information, typically used during development and debugging.
# INFO: General information, such as indicating that the program has started or completed an operation.
# WARNING: An indication that something unexpected happened or indicative of some problem in the near future (e.g., ‘disk space low’). The software is still working as expected.
# ERROR: Due to a more serious problem, the software has not been able to perform some function.
# CRITICAL: A very serious error, indicating that the program itself may be unable to continue running.

In [ ]:
# Since the given config is used to train PSPNet on the cityscapes dataset, we need to modify it accordingly for our new dataset.
# Load the pretrained weights
cfg.load_from = checkpoint_path
# cfg.load_from = None

# cfg.model.backbone.init_cfg=dict(type='Pretrained', checkpoint=checkpoint_path)
#set batch normalisation settings
# Since we use only one GPU, BN is used instead of SyncBN . 
# cfg.model.backbone.norm_cfg is kept as is , causes an error if set to BN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

#define paths for imgs and annotations
data_prefix_dict = dict(img_path=img_dir_path, seg_map_path=ann_dir_path)

#set segmentation index to ignore - class 0 = outside ROI
cfg.model.decode_head.ignore_index = 0
cfg_orig.model.decode_head.loss_decode.avg_non_ignore = True #we don't want to take into account ignored seg indexes when calculating losses

# modify num classes of the model in decode/auxiliary head
cfg.num_classes = 6 #for mask2former
cfg.model.decode_head.num_classes = 6  #6 total classes minus ignored class of 0
# cfg.model.auxiliary_head.num_classes = 6

#set model.pretrained to None as we have already loaded the checkpoints
cfg.model.pretrained = None

class_weights = [1,1,1,1,1,1]

# create custom loss
cfg.model.decode_head.loss_decode=[
        # dict(type='DiceLoss', loss_name='loss_dice', loss_weight=1.0,class_weight=class_weights),
        dict(type='CrossEntropyLoss', loss_name='loss_ce', loss_weight=1.0,class_weight=class_weights),
        # dict(type='LovaszLoss',loss_name='loss_lov',loss_type='multi_class',
        #          classes='present',per_image=False,
        #          reduction='none', loss_weight=1.0, class_weight=class_weights)
        # dict(type='FocalLoss',loss_weight=1.0)
        # dict(type='TverskyLoss')
        
    ]

# cfg.model.test_cfg.mode='slide'


# cfg.model.decode_head.sampler = dict(type='OHEMPixelSampler', thresh=0.7, min_kept=200000)

# cfg.img_ratios = [0.75, 1.0, 1.25]
FINAL_IMG_SIZE = (512,512)

data_preprocessor = dict(
    type='SegDataPreProcessor',
    # mean=[0.485, 0.456, 0.406],#
    # std=[0.229, 0.224, 0.225],
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=FINAL_IMG_SIZE)

cfg.data_preprocessor = data_preprocessor
cfg.model.data_preprocessor = data_preprocessor

CROP_SIZE = (768,768)


RESIZE_SIZE = FINAL_IMG_SIZE
cfg.crop_size = CROP_SIZE
# cfg.data_preprocessor = None
# cfg.model.data_preprocessor = None
# cfg.data_preprocessor.size=FINAL_IMG_SIZE
# cfg.model.data_preprocessor.si=FINAL_IMG_SIZE

# Modify dataset type and path
cfg.dataset_type = 'CancerDataset'
cfg.data_root = data_root

#set some hyperparameter values for training
BATCH_SIZE = 4
NUM_WORKERS = 1
USE_PERS_WORKERS = False

#define the pipeline for loading training images and masks
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', reduce_zero_label=False),
    dict(type='PackSegInputs')
]

#define the pipeline for loading validatrion/test images and masks
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    # dict(type='Resize', scale=FINAL_IMG_SIZE),
    # add loading annotation after ``Resize`` because ground truth
    dict(type='LoadAnnotations'),
    # does not need to do resize data transform
    dict(type='PackSegInputs')
]


#define variables for loading training images and masks - normal
# cfg.train_dataloader.dataset.pop('times')
# cfg.train_dataloader.dataset.pop('dataset')
# cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
# cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset=dict(
        type='RepeatDataset',
        times=1,
        dataset=dict(
            type='CancerDataset',
            data_root=cfg.data_root,
            data_prefix=data_prefix_dict,
            pipeline=cfg.train_pipeline,
            ann_file='splits/train.txt'))

# cfg.train_dataloader.dataset.data_root = cfg.data_root
# cfg.train_dataloader.dataset.data_prefix = data_prefix_dict
# cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'
cfg.train_dataloader.batch_size = 2
cfg.train_dataloader.num_workers= 1
# cfg.train_dataloader.persistent_workers=True
cfg.train_dataloader.sampler = dict(type='InfiniteSampler', shuffle=True)
# cfg.train_dataloader.pin_memory=True

#poolformer
# cfg.train_dataloader.dataset.times = 10
# cfg.train_dataloader.dataset.dataset.pipeline = cfg.train_pipeline
# cfg.train_dataloader.dataset.dataset.type = cfg.dataset_type
# cfg.train_dataloader.dataset.dataset.data_root = cfg.data_root
# cfg.train_dataloader.dataset.dataset.data_prefix = data_prefix_dict
# cfg.train_dataloader.dataset.dataset.ann_file = 'splits/train.txt'
# cfg.train_dataloader.batch_size = BATCH_SIZE
# cfg.train_dataloader.num_workers= NUM_WORKERS
# cfg.train_dataloader.persistent_workers=USE_PERS_WORKERS
# cfg.train_dataloader.sampler = dict(type='InfiniteSampler', shuffle=True)


#define variables for loading validation images and masks
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = data_prefix_dict
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'
cfg.val_dataloader.batch_size = 1
cfg.val_dataloader.num_workers=4
cfg.val_dataloader.persistent_workers=USE_PERS_WORKERS
cfg.val_dataloader.sampler = dict(type='DefaultSampler', shuffle=False)

#define variables for loading training images and masks
cfg.test_dataloader = cfg.val_dataloader
# cfg.model.test_cfg.mode='slide'
# cfg.model.test_cfg.crop_size=(512,512)
# cfg.model.test_cfg.stride = (256,256)

# Set up working dir to save files and logs.
cfg.work_dir = '/content/gdrive/MyDrive/BCSS_dataset/logs'

max_iters = 2000

#define model training parameters
cfg.train_cfg.max_iters = max_iters #max number of iterations
cfg.train_cfg.val_interval = 500 #number of iterations to check model against validation dataset
# cfg.train_cfg.gradient_accumulation_steps=4
# cfg.train_cfg.amp_cfg=dict(opt_level='O1')
cfg.fp16 = dict(opt_level='O1') #was 01 = amp , 03 = forced 16
cfg.default_hooks.logger.interval = 100 #number of iteratios to print status
cfg.default_hooks.checkpoint.interval = 0 #how often to save checkpoint of the model 
cfg.log_level = 'INFO' #logging level during training , was'INFO'
cfg.log_processor = dict(by_epoch=False)
# cfg.custom_hooks=[dict()]

# define learning rate scheduler and others
cfg.param_scheduler = [
    # dict(
    #     type='LinearLR', start_factor=1e-06, by_epoch=False, begin=0,
    #     end=500), #was 1500
    # dict(
    #     type='PolyLR', #possibly change to 1cycle
    #     eta_min=0.0005,
    #     power=1.0,
    #     begin=500,#was 1500
    #     end=max_iters,
    #     by_epoch=False)

    dict(
        type='OneCycleLR', #possibly change to 1cycle
        eta_max=0.001,
        total_steps = max_iters,
        three_phase = True,
        by_epoch=False),

    # dict(
    #     type='ConstantLR', #possibly change to 1cycle
    #     factor=0.001,
    #     by_epoch=False),

    # dict(
    #     type='LinearMomentum',
    #      start_factor = 1.0,
    #      end_factor = 0.6,
    #      begin=0,
    #      end = int(max_iters/2),
    #      by_epoch=False),
    
    #  dict(
    #     type='LinearMomentum',
    #      start_factor = 0.6,
    #      end_factor = 1.0,
    #      begin=int(max_iters/2),
    #      end = max_iters,
    #      by_epoch=False)
    ]
    
  
    

# Set seed to facilitate reproducing the result


#set optimizer - change to adamw
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
optim_wrapper = dict(
    type='AmpOptimWrapper',
    optimizer=dict(type='AdamW', lr=6e-05, betas=(0.9, 0.999), weight_decay=0.01),
    paramwise_cfg=dict(
      custom_keys=dict(
          pos_block=dict(decay_mult=0.0),
          norm=dict(decay_mult=0.0),
          head=dict(lr_mult=10.0))))

# cfg.optim_wrapper = dict(
#     type='AmpOptimWrapper',
#     optimizer=cfg.optimizer,
#     paramwise_cfg = dict(
#       custom_keys={
#           'head': dict(lr_mult=5.)}))


cfg.tta_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='TestTimeAug',
        transforms=[[{
            'type': 'Resize',
            'scale_factor': 0.5,
            'keep_ratio': True
        }, {
            'type': 'Resize',
            'scale_factor': 0.75,
            'keep_ratio': True
        }, {
            'type': 'Resize',
            'scale_factor': 1.0,
            'keep_ratio': True
        }, {
            'type': 'Resize',
            'scale_factor': 1.25,
            'keep_ratio': True
        }, {
            'type': 'Resize',
            'scale_factor': 1.5,
            'keep_ratio': True
        }],
                    [{
                        'type': 'RandomFlip',
                        'prob': 0.5,
                        'direction': 'vertical'
                    }, {
                        'type': 'RandomFlip',
                        'prob': 0.5,
                        'direction': 'horizontal'
                    }], [{
                        'type': 'LoadAnnotations'
                    }], [{
                        'type': 'PackSegInputs'
                    }]])
]
        

# set metrics to use during evaluation
cfg.val_evaluator = dict(type='IoUMetric',iou_metrics=['mDice','mIoU'])
cfg.test_evaluator = dict(type='IoUMetric', iou_metrics=['mDice'])

# cfg.compile = True
cfg.tta_model = None

# check final config
# print(f'Config:\n{cfg.pretty_text}')

#mask2former
# cfg.model.decode_head.loss_cls.class_weight = class_weights

cfg['randomness'] = dict(seed=19) #0,15,42,180

### Combined augments

This section has a proposed image augmentation pipeline

In [ ]:
#color augmentations
colour_augments = dict(type='OneOf',
            transforms=[
              dict(type='HueSaturationValue',
                hue_shift_limit=30,#was 20, 
                sat_shift_limit=10,# was30, 
                val_shift_limit=50,
                p=0.2),

              dict(type='RandomGamma',
                gamma_limit = (50,100),
                p=1.0),

              dict(type='ColorJitter',
                  brightness=0.9,
                  contrast=0.3,
                  saturation=0.5,
                  hue=0.3,
                  p=0.3),
              
              dict(type='Equalize',
                   mode='cv', by_channels=True,
                    p=1.0)

            ],p=0.4)

#choose betweeen spatial transforms
spatial_transforms=dict(type='OneOf',
              transforms=[
                dict(type='OpticalDistortion',
                  distort_limit=[-0.35,0.2],
                  shift_limit=[0,0.3],
                  p=1.0),

              dict(type='GridDistortion',
                distort_limit=[-0.1,0.3],
                steps=8,
                p=1.0),

              dict(type='ElasticTransform',
                alpha= 750, #0-1000 or 500-1000 common #was 1000
                sigma= 22, #4-10 common
                alpha_affine= 35, #0 - 50 common 
                interpolation=1, 
                border_mode=2, 
                value=255, mask_value=0, 
                approximate=False, same_dxdy=False, p=3),

            ],p=0.4)


#choose between clarity transforms
clarity_transforms=dict(type='OneOf',
            transforms=[
              dict(type='GaussianBlur',
                blur_limit=6,
                sigma_limit=5,
                p=0.3),

              #sharpen
              dict(type='Sharpen',
                    alpha=0.7, 
                     lightness=5,
                      p=1.0),

              #emboss
              dict(type='Emboss',
                    alpha=0.7,
                    emboss_strength=1.2,
                    p=1.0)

            ],p=0.3)


scale_down = dict(type='Sequential',
                  transforms=[
                    dict(type='RandomScale',
                      scale_limit=(-0.3, 0),p=1.0),
                    dict(type='PadIfNeeded',
                        min_height=512,
                        min_width=512,
                        p=1.0)],
                  p=1.0)

scale_up = dict(type='Sequential',
                  transforms=[
                    dict(type='RandomScale',
                          scale_limit=(0, 0.3),p=1.0),
                    dict(type='RandomCrop',
                        height=512, 
                        width=512,
                        p=1.0)
                          ],p=1.0)


choose_scale_up_or_down = dict(type='OneOf',
                            transforms=[scale_up,
                              scale_down]
                            ,p=0.3)


      

combined_transforms = dict(type='Sequential',
                        transforms=[colour_augments,
                                    spatial_transforms,
                                    clarity_transforms],
                                    # choose_scale_up_or_down],
                                  
                          p=0.5)

final_resize = dict(type='Resize',
                  height=FINAL_IMG_SIZE[0],
                  width=FINAL_IMG_SIZE[1],
                  p=1.0),

rand_rotate=dict(type='RandomRotate90',p=0.5)             
          
grid_drop = dict(type='GridDropout',
                  ratio=0.5, 
                  unit_size_min=2, 
                  unit_size_max=10, 
                  holes_number_x=32, holes_number_y=32, 
                  shift_x=0, shift_y=0, random_offset=True, fill_value=0, mask_fill_value=None, 
                  always_apply=False, p=0.2)

final = [dict(type='Sequential',
          transforms=[
              combined_transforms,
              final_resize,
              rand_rotate,
              # grid_drop
          ],p=1.0)]

      

Below is config file from previous iteration , not currently used but can be checked for inspiration on which things to change to improve model performance

In [ ]:
# !rm /content/gdrive/MyDrive/BCSS_dataset/logs -r
# !mkdir /content/gdrive/MyDrive/BCSS_dataset/logs

#### Show final config

In [ ]:
cfg.train_pipeline = [
      dict(type='LoadImageFromFile'),
      dict(type='LoadAnnotations',reduce_zero_label=False),
      dict(type='RandomCrop',crop_size=CROP_SIZE),
      # dict(type='Albu',transforms=[final_resize]),
      dict(type='Albu', transforms=final),
      # dict(type='Resize',scale=FINAL_IMG_SIZE),
      dict(type='PackSegInputs')]

print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/poolformer/poolformer-m48_3rdparty_32xb128_in1k_20220414-9378f3eb.pth'
custom_imports = dict(imports=['mmcls.models'], allow_failed_imports=False)
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=(512, 512))
model = dict(
    type='EncoderDecoder',
    data_preprocessor=dict(
        type='SegDataPreProcessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_val=0,
        seg_pad_val=255,
        size=(512, 512)),
    backbone=dict(
        type='mmcls.PoolFormer',
        arch='m48',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/poolformer/poolformer-m48_3rdpar

In [ ]:
from datetime import datetime

def get_tumor_dice_from_logs(log_path):
  for line in reversed(list(open(log_path))):
    if 'tumor' in line:
      tum_dice_score = float(line.split('|')[2].strip(' '))
      return tum_dice_score
  
  print('dice score not found in log file')
  return 1

def get_most_recent_log_path():
  log_dir = '/content/gdrive/MyDrive/BCSS_dataset/logs'
  files = [os.path.join(log_dir,file_name) for file_name in os.listdir(log_dir)]
  # print(files)
  files_sorted = sorted(files,key=os.path.getctime)
  print(files_sorted)
  log_days_hours = [f.split('/')[-1] for f in files_sorted if f.split('/')[-1][0:4]=='2023']
  
  most_recent = None
  for log_file in log_days_hours:
    # Parse the date and time from the log file name
    log_date, log_time = log_file.split("_")

    # Convert the date and time strings to datetime objects
    log_datetime = datetime.strptime(f"{log_date}_{log_time}", "%Y%m%d_%H%M%S")

    # Check if this log file is more recent than the current most recent
    if most_recent is None or log_datetime > most_recent:
        most_recent = log_datetime

  # print(most_recent)
  most_recent_string = most_recent.strftime("%Y%m%d_%H%M%S")
  # print(most_recent_string)
  for f in files_sorted:
    if str(most_recent_string) in f:
      log_folder = f
      break

  log_path = os.path.join(log_folder,most_recent_string)+'.log'
  # print(log_path)
  return log_path


# get_most_recent_log_path()
print(get_most_recent_log_path())
print(get_tumor_dice_from_logs(get_most_recent_log_path()))

['/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_125015', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_125449', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_125714', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_130040', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_130523', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_220904', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230510_222405', '/content/gdrive/MyDrive/BCSS_dataset/logs/fpn_poolformer_s24_8xb4-40k_ade20k-512x512.py', '/content/gdrive/MyDrive/BCSS_dataset/logs/iter_8000.pth', '/content/gdrive/MyDrive/BCSS_dataset/logs/iter_12000.pth', '/content/gdrive/MyDrive/BCSS_dataset/logs/iter_14000.pth', '/content/gdrive/MyDrive/BCSS_dataset/logs/iter_16000.pth', '/content/gdrive/MyDrive/BCSS_dataset/logs/iter_18000.pth', '/content/gdrive/MyDrive/BCSS_dataset/logs/iter_20000.pth', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230511_074705', '/content/gdrive/MyDrive/BCSS_dataset/logs/20230511_075809', 

In [ ]:
break_run_all

NameError: ignored

### Optuna

In this section we define the image augmentation to be opimised and select upper and lower limits as well as step size for each augmentation parameter in the search space. Each experiment runs for 20 trials. 

In [ ]:
from mmengine.runner import Runner
fp16_cfg = cfg.get('fp16', None)
def objective(trial):

  # gamma_limit_lower = trial.suggest_float("gamma_blower",low=0,high=150,step=30,log=False)
  # gamma_limit_delta = trial.suggest_float("gamma_delta",low=0,high=50,step=10,log=False)
  # gamma_limit_upper = gamma_limit_lower + gamma_limit_delta

  # clahe_tile_grid_size = trial.suggest_int("grid_size",low=8,high=64,step=8)

  # contrast_limit_lower = trial.suggest_float("contrast_limit_lower",low=-1.0,high=1.0,step=0.2,log=False)
  # contrast_limit_upper = trial.suggest_float("contrast_limit_upper",low=-1.0,high=1.0,step=0.2,log=False)
  #define augmentation search space

  # max_cutout_size = trial.suggest_int("max_size",low=0,high=20,step=2)
  # holes_no = trial.suggest_int("max_size",low=0,high=100,step=20)
  # unit_size_min=trial.suggest_int("unit_min",low=10,high=210,step=20,log=False)
  # unit_size_delta = trial.suggest_int("unit_max",low=10,high=210,step=20,log=False)

  # distort_limit_low = trial.suggest_float("distort_limit_low", low=-0.5, high=0.0, step=0.05)
  # distort_limit_high = trial.suggest_float("distort_limit_high", low=0, high=0.5, step=0.05)

  # shift_limit_low = trial.suggest_float("shift_limit_low", low=-0.5, high=0.0, step=0.05)
  # shift_limit_high = trial.suggest_float("shift_limit_high", low=0, high=0.5, step=0.05)

  transforms = [
      
      
      # dict(type='CLAHE',
      #     clip_limit=trial.suggest_int("clip_limit",low=0,high=10,step=1), 
      #     tile_grid_size=(clahe_tile_grid_size, clahe_tile_grid_size),
      #     p=0.5)

      # dict(type='RandomGamma',
      #      gamma_limit = (gamma_limit_lower,gamma_limit_upper),
      #      p=0.5)

      # dict(type='CutOut',
      #      num_holes=trial.suggest_int("n_holes",low=0,high=20,step=2),
      #      max_h_size=max_cutout_size,
      #      max_w_size=max_cutout_size,
      #      fill_value=255,
      #      p=0.5)

      # dict(type='HueSaturationValue',
      #  hue_shift_limit=trial.suggest_int("hue_limit",low=0,high=50,step=10,log=False),#was 20, 
      #  sat_shift_limit=trial.suggest_int("sat_limit",low=0,high=50,step=10,log=False),# was30, 
      #  val_shift_limit=trial.suggest_int("val_limit",low=0,high=50,step=10,log=False),
      #  p=0.5)

      # dict(type='ColorJitter',
      #     brightness=trial.suggest_float("color_jit_bright",low=0.1,high=0.9,step=0.2,log=False),
      #     contrast=trial.suggest_float("color_jit_contrast",low=0.1,high=0.9,step=0.2,log=False),
      #     saturation=trial.suggest_float("color_jit_sat",low=0.1,high=0.9,step=0.2,log=False),
      #     hue=trial.suggest_float("color_jit_hue",low=0.1,high=0.9,step=0.2,log=False),
      #     p=0.5
      #     )

      # dict(type='ElasticTransform',
      #      alpha= trial.suggest_int("alpha",low=0,high=1000,step=100,log=False), #0-1000 or 500-1000 common
      #      sigma= trial.suggest_int("sigma",low=0,high=20,step=2,log=False), #4-10 common
      #      alpha_affine= trial.suggest_int("alpha_affine",low=0,high=60,step=10,log=False), #0 - 50 common 
      #      interpolation=1, 
      #      border_mode=cv2.BORDER_CONSTANT, 
      #      value=255, mask_value=0, 
      #      approximate=True, same_dxdy=False, p=0.5)

      # dict(type='OpticalDistortion',
      #   distort_limit=[distort_limit_low,distort_limit_high],
      #   shift_limit=[shift_limit_low,shift_limit_high],
      #   p=0.5)

      # dict(type='GridDistortion',
      #   num_steps=trial.suggest_int("num_steps", low=4, high=20, step=2),
      #   distort_limit=(distort_limit_low,distort_limit_high),
      #   p=0.5)

      # dict(type='RandomContrast',
      #      limit=[contrast_limit_lower,contrast_limit_upper],
      #      p=0.5)
      
      # dict(type='ChannelShuffle',
      #     p=0.5),
      
      # dict(type='Emboss',
      #     alpha=(0.2, 0.5),
      #     strength=(0.2, 0.7),
      #     p=0.5)
      # dict(tye='Resize',height=FINAL_IMG_SIZE[0],width=FINAL_IMG_SIZE[1]),
      # dict(type='GridDropout',ratio=0.5, 
      #           unit_size_min=unit_size_min, 
      #           unit_size_max=unit_size_min+unit_size_delta, 
      #           holes_number_x=holes_no, holes_number_y=holes_no, shift_x=0, shift_y=0, random_offset=False, fill_value=0, mask_fill_value=None, always_apply=False, p=0.5)
      
      # dict(type='GaussianBlur',
      #   blur_limit=trial.suggest_int("blur_limit", low=0, high=10, step=1),
      #   sigma_limit=trial.suggest_float("sigma_limit", low=0.0, high=5.0, step=0.5),
      #   p=0.5)

      # dict(type='Sharpen', 
      #   alpha=trial.suggest_float("sharpen_alpha", low=0.0, high=1.0, step=0.1), 
      #   lightness=trial.suggest_int("sharpen_lightness", low=0, high=10, step=1),
      #   p=0.5)

    #   dict(type='Emboss',
    #  alpha=trial.suggest_float("emboss_alpha", low=0.0, high=1.0, step=0.1), 
    #  strength=trial.suggest_float("emboss_strength", low=0.0, high=2.0, step=0.1),
    #  p=0.5)


  ]


  cfg.train_pipeline = [
      dict(type='LoadImageFromFile'),
      dict(type='LoadAnnotations',reduce_zero_label=False),
      # dict(type='Resize', scale=FINAL_IMG_SIZE),#, ratio_range=(0.5, 2.0), keep_ratio=True)
      dict(type='Albu', transforms=final),
      
  #     # dict(type='PhotoMetricDistortion'),
  #     # ,
  #     # dict(type='RandomFlip', prob=0.5),
  #     # dict(type='Rerange', min_value=0, max_value=1),
      dict(type='PackSegInputs')]

  runner = Runner.from_cfg(cfg) 
  runner.train()

  trial_dice_score = get_tumor_dice_from_logs(get_most_recent_log_path())

  return trial_dice_score

study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=1, timeout=None)

print('finished study')




In [ ]:
from optuna.trial import TrialState
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



In [ ]:
def gen_df(study_obj):
  trial_df = study_obj.trials_dataframe()
  trial_df_new = trial_df.drop(columns=['number','datetime_start','datetime_complete','duration','state'])
  trial_df_new = trial_df_new.sort_values(by='value',ascending=False)
  return trial_df_new

df_out = gen_df(study)
# df_out = df_out.rename(columns={'params_gamma_blower':'params_gamma_lower'}
df_out.to_csv('/content/gdrive/MyDrive/BCSS_dataset/optuna_results/Emboss.csv')
print(df_out.describe())
df_out.head(20)

# 

In [ ]:
import pandas as pd
df_out = pd.read_csv('/content/gdrive/MyDrive/BCSS_dataset/optuna_results/GridDistortion.csv')
df_out.head(20)

In [ ]:
from statistics import mean


def get_avg_tumor_dice_from_logs(log_path,expected_scores):
  print(f'checking {log_path}')
  dice_scores = []
  scores_found = 0
  for line in reversed(list(open(log_path))):
    if 'tumor' in line:
      tum_dice_score = float(line.split('|')[2].strip(' '))
      dice_scores.append(tum_dice_score)
      scores_found += 1
    
    if scores_found == expected_scores:
      for x in reversed(dice_scores):
        print(f"{x}")
      return mean(dice_scores)

print(get_avg_tumor_dice_from_logs(get_most_recent_log_path(),4))